Import and Path

In [1]:
#* General imports
import os, sys
import numpy as np 
from pathlib import Path 
import pandas as pd
from tqdm import tqdm



path_to_mgr = Path('/projects/academic/kaihangs/czheng37/Reproduction-0705/Codes_Organization/mofography_poronet')
#path_to_mgr = Path('/projects/academic/kaihangs/czheng37/CCR/mofography-latest-1021')
# Chao's modified functions are added locally in a code cell. 
sys.path.append(str(path_to_mgr))

import pickle
import mofography as mgr

#* Needed for cheminformatics if any
from ase.io import read, write
from ase.build import make_supercell 
from ase.geometry import complete_cell


#* for plotting
import plotly.graph_objects as go

#* Needed for the workflow
import dask
from dask import delayed
from dask.distributed import Client, progress, LocalCluster

from skimage.measure import regionprops

Load the dataset

In [2]:
path_to_df = Path('/projects/academic/kaihangs/czheng37/Reproduction-0705/Codes_Organization/Selected_MOFs_for_H2.feather')
df_Tobacco_training = pd.read_feather(path_to_df)

In [3]:
Tobacco_atoms = mgr.atoms_from_mofdb_df(df_Tobacco_training[0:10])

  0%|          | 0/10 [00:00<?, ?it/s]/projects/academic/kaihangs/czheng37/anaconda3/envs/pgcnn/lib/python3.11/site-packages/ase/io/cif.py:401: UserWarning: crystal system 'triclinic' is not interpreted for space group Spacegroup(1, setting=1). This may result in wrong setting!
  warnings.warn(
100%|██████████| 10/10 [00:00<00:00, 11.76it/s]


Create a workflow for a single material

Let's add your modified functions here, and replace the usage `mgr.your_function_name` with just `function_name` in the cell below for testing purposes. 

In [4]:
def regions_maxima_radii_from_dgrid(dgrid, mask_thickness,h, threshold_abs):
    regions, maxima = mgr.regions_from_dgrid_with_threshold_abs(dgrid, mask_thickness,h,threshold_abs)
    maxima_radii = dgrid[tuple(maxima.T )]
    print('made the regions and maxima')
    return regions, maxima, maxima_radii

def make_rag(regions, maxima, maxima_radii, dgrid, mat_atoms,spacing):
    connections = mgr.connections_from_regions_and_dgrid(regions, dgrid, maxima, mat_atoms)
    rag = mgr.rag_from_connections_pixel_multi(regions,connections, maxima, only_use_internal=False)
    rag  = mgr.add_pixel_ratio_to_rag(rag, mat_atoms, spacing) 
    shape = dgrid.shape
    rag = mgr.add_maxima_to_rag(rag, maxima, maxima_radii, shape, mat_atoms)
    rag = mgr.add_volume_to_rag(rag,mat_atoms)
    print('made the rag')
    return rag

def energy_calculation(mat_atoms, spacing, blocksize,forcefield_mixed,cutoff,probe_symbol):
    egrid = mgr.egrid_from_atoms2(mat_atoms, spacing, blocksize, forcefield_mixed, cutoff, probe_symbol, return_ncells=False, precompute_aabb=False)
    egrid= mgr.egrid_transfer(egrid)
    einterp=mgr.get_energy_interpolator(egrid)
    print('made the einterp')
    return einterp

def energy_histogram(rag,regions,einterp,pbc_groups):
    bins1=np.array([-np.inf,-10])
    bins2=np.linspace(-9,-1,9)
    bins3 = np.array([0, np.inf])
    bins=np.concatenate((bins1,bins2,bins3))
    rag=mgr.add_vdw_hist_to_rag_probability_right(rag, regions, einterp, use_pbc=True, pbc_groups=pbc_groups, bins_energy=bins)
    print('made the bins and added ehist')
    return rag


In [5]:
bins1=np.array([-np.inf,-10])
bins2=np.linspace(-9,-1,9)
bins3 = np.array([0, np.inf])
bins=np.concatenate((bins1,bins2,bins3))

In [6]:
bins

array([-inf, -10.,  -9.,  -8.,  -7.,  -6.,  -5.,  -4.,  -3.,  -2.,  -1.,
         0.,  inf])

In [7]:
#grid=0.5
delayed_rag_futures = []
for mat_atoms in Tobacco_atoms:
    dgrid = delayed(mgr.dgrid_from_atoms_cpu_no_aabb)(mat_atoms, spacing=0.5)
    regions_maxima_radii = delayed(regions_maxima_radii_from_dgrid)(dgrid, mask_thickness=0, h=0.5, threshold_abs=1)
    rag = delayed(make_rag)(regions=regions_maxima_radii[0], maxima=regions_maxima_radii[1],maxima_radii=regions_maxima_radii[2], dgrid=dgrid, mat_atoms=mat_atoms, spacing=0.5)
    pbc_groups = delayed(mgr.apply_pbc)(regions_maxima_radii[0], regions_maxima_radii[1], regions_maxima_radii[2], mat_atoms, return_conn=True)
    rag = delayed(mgr.add_pbc_to_rag)(rag,pbc_groups)
    einterp=delayed(energy_calculation)(mat_atoms,spacing=0.5, blocksize=20000, forcefield_mixed=None,cutoff=12.8,probe_symbol='H_com')
    rag = delayed(energy_histogram)(rag,regions=regions_maxima_radii[0],einterp=einterp,pbc_groups=pbc_groups)
    delayed_rag_futures.append(rag)



We don't typically need that many workers to compute rags for say a few hundred or thousand materials. 

So can even simply do this,

In [8]:
# client = Client()
# cient.shutdown()

or this , just make suer you have enough RAM (5 GB) for each of the workers, just to be safe.

In [9]:
client = Client(n_workers=64)

In [10]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 64
Total threads: 64,Total memory: 488.28 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:36733,Workers: 64
Dashboard: http://127.0.0.1:8787/status,Total threads: 64
Started: Just now,Total memory: 488.28 GiB
Comm: tcp://127.0.0.1:42813,Total threads: 1
Dashboard: http://127.0.0.1:45475/status,Memory: 7.63 GiB
Nanny: tcp://127.0.0.1:45583,


In [11]:

# cluster = LocalCluster(n_workers=64, threads_per_worker=1)
# client = Client(cluster)
# print('THe client is running on : ', client.dashboard_link)

In [12]:
#* submit the workflow using the futures API
futures = dask.persist(*delayed_rag_futures) 

In [13]:
progress(futures)

VBox()

/projects/academic/kaihangs/czheng37/anaconda3/envs/pgcnn/lib/python3.11/site-packages/ase/utils/__init__.py:62: FutureWarning: Please use atoms.cell.cellpar() instead
  warnings.warn(warning)
/projects/academic/kaihangs/czheng37/anaconda3/envs/pgcnn/lib/python3.11/site-packages/ase/utils/__init__.py:62: FutureWarning: Please use atoms.cell.cellpar() instead
  warnings.warn(warning)
/projects/academic/kaihangs/czheng37/anaconda3/envs/pgcnn/lib/python3.11/site-packages/ase/utils/__init__.py:62: FutureWarning: Please use atoms.cell.cellpar() instead
  warnings.warn(warning)
/projects/academic/kaihangs/czheng37/anaconda3/envs/pgcnn/lib/python3.11/site-packages/ase/utils/__init__.py:62: FutureWarning: Please use atoms.cell.cellpar() instead
  warnings.warn(warning)
/projects/academic/kaihangs/czheng37/anaconda3/envs/pgcnn/lib/python3.11/site-packages/ase/utils/__init__.py:62: FutureWarning: Please use atoms.cell.cellpar() instead
  warnings.warn(warning)
/projects/academic/kaihangs/czheng3

made the regions and maxima
made the regions and maxima
made the einterp
made the einterp


ic| regions_to_check: array([1, 2, 3, 4, 5, 6, 7, 8], dtype=int32)
ic| len(outer): 8
ic| regions_to_check: array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16],
                            dtype=int32)
ic| len(outer): 16


made the einterp
made the regions and maxima
made the rag


...

made the rag
made the regions and maxima


ic| regions_to_check: array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
                             18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28], dtype=int32)


made the bins and added ehist


/projects/academic/kaihangs/czheng37/anaconda3/envs/pgcnn/lib/python3.11/site-packages/ase/utils/__init__.py:62: FutureWarning: Please use atoms.cell.cellpar() instead
  warnings.warn(warning)
ic| len(outer): 28
/projects/academic/kaihangs/czheng37/anaconda3/envs/pgcnn/lib/python3.11/site-packages/ase/utils/__init__.py:62: FutureWarning: Please use atoms.cell.cellpar() instead
  warnings.warn(warning)


made the rag
made the bins and added ehist


### Do not shutdown the client before colecting results
`client.shutdown()` is for shutting down the client and it will clear all workers and you'll loose the data. 

In [ ]:
client.shutdown()  # stop

In [14]:
results = client.compute(futures)

In [15]:
rags = [r.result() if r.status == 'finished' else None for r in results]

In [16]:
len(rags)

10

In [17]:
rags[0:1000]

In [ ]:
rag[0]

In [ ]:
rags[1000:2000]

In [ ]:
np.sum([a == None for a in rags])

In [ ]:
rag_list = []
none_indices = []  # 用于存储 None 元素的索引

for i, g in enumerate(rags):
    if g is None:
        none_indices.append(i)  # 记录 None 的索引
    else:
        rag_list.append(g)  # 仅保留非 None 元素



In [ ]:
none_indices

In [ ]:
rag_list[0].number_of_nodes()

In [ ]:
len(rag_list)

In [ ]:
rag_list

In [ ]:
pickle.dump(rag_list, open(Path('/projects/academic/kaihangs/czheng37/Reproduction/Graphs/H2/H2-0609.pkl'), 'wb'))

## Summary
- Tested on laptop with 8 cores nad 16 gb RAM. should work fine on CCR.
- Do not call `client.shutdown` before gathering the results
- `total RAM / n_workers >= 10 GB` should be good. choose n_workers accordingly
- add enough threads per worker so that parallelization will be effective, otherwise each individual calculation might take forever. better off setting `n_workers` instead.
- the lines like `[energy_histogram_right_inclusive(e, bins=bins_energy, weights=1/len(e)) for e in energies_list] ` might be too slow for large MOFs
- if passing the energy interpolators in the task graph slows things down, might have to consider calculating all at once and scattering them beforehand. We can worry about it later.